In [1]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=1
#%matplotlib inline

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from skimage.measure import compare_psnr
import tensorflow as tf
from data.dataset import Dataset
import data.data_selector as ds
import utils.data_processing as dp
import utils.plot_functions as pf
import analysis.analysis_picker as ap

/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [3]:
[name[17:-4] for name in os.listdir(os.path.expanduser("~/Work/Projects/lca_subspace_vh/logfiles"))]

['5x_3_2.0_0.2',
 '5x_4_2.0_0.2',
 '8_3.0_0.3',
 '5x_5_1.0_0.2',
 '4_3.0_0.5',
 '8_4.0_0.3',
 '5x_4_1.5_0.2',
 '4_3.0_0.3',
 '4_3.0_0.4',
 '4_3.0_0.2',
 '8_3.0_0.4',
 '5x_3_1.0_0.2',
 '5x_2_1.5_0.2',
 '5x_2_2.0_0.2',
 '4_5.0_0.3',
 '8_5.0_0.3',
 '8_4.0_0.5',
 '8_5.0_0.2',
 '4_5.0_0.2',
 '5x_4_1.75_0.2',
 '5x_2_1.0_0.2',
 '5x_4_2.0_0.3',
 '5x_4_1.0_0.2',
 '5x_4_3.0_0.3',
 '5x_4_1.25_0.2',
 '5x_5_1.5_0.2',
 '10x_4_3.0_0.3',
 '5x_4_1.0_0.1',
 '5x_3_1.5_0.2',
 '8_4.0_0.2',
 '8_3.0_0.5',
 '8_3.0_0.2',
 '5x_4_0.5.0_0.2',
 '8_4.0_0.4',
 '4_3.0_0.3_5x',
 '5x_4_2.0_0.1',
 '4_4.0_0.4',
 '16_5.0_0.4',
 '8_5.0_0.4',
 '4_4.0_0.3',
 '5x_5_2.0_0.2',
 '4_4.0_0.2']

## Prelimiaries

In [4]:
class params:
    def __init__(self):
        self.model_type = "lca_subspace"
        self.model_name = "lca_subspace_vh"
        self.version = "5x_4_1.0_0.2"
        self.save_info = "analysis_train"
        self.overwrite_analysis_log = False
        
analysis_params = params()
analysis_params.project_dir = (os.path.expanduser("~")+"/Work/Projects/")
analysis_params.model_dir = (analysis_params.project_dir+analysis_params.model_name)

In [5]:
analyzer = ap.get_analyzer(analysis_params.model_type)
analyzer.setup(analysis_params)
analyzer.setup_model(analyzer.model_params)
analyzer.load_analysis(save_info=analysis_params.save_info)
analyzer.model_name = analysis_params.model_name

FileNotFoundError: [Errno 2] No such file or directory: '/home/ryanchan/Work/Projects/lca_subspace_vh/analysis/5x_4_1.0_0.2/savefiles/co_responses_analysis_train.npz'

In [ ]:
dataset = ds.get_data(analyzer.model_params)
dataset = analyzer.model.preprocess_dataset(dataset, analyzer.model_params)
dataset = analyzer.model.reshape_dataset(dataset, analyzer.model_params)
target_image = dataset["train"].images[0,...]

In [ ]:
num_imgs = 100#data["train"].images.shape[0]
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=analyzer.model.graph) as sess:
  feed_dict = analyzer.model.get_feed_dict(dataset["train"].images[0:num_imgs,...])
  sess.run(analyzer.model.init_op, feed_dict)
  analyzer.model.load_full_model(sess, analyzer.analysis_params.cp_loc)
  run_list = [analyzer.model.module.w, analyzer.model.a, analyzer.model.module.group_activity, analyzer.model.module.group_angles]
  weights, neuron_activations, group_activations, group_angles = sess.run(run_list, feed_dict)
  weights_reshaped = weights.T.reshape(analyzer.model_params.num_neurons, analyzer.model_params.patch_edge_size, analyzer.model_params.patch_edge_size)

## population statistics

In [ ]:
neuron_ids = list(range(16))
contrast = 1.0
gauss_sigma = 1.0
orientation = None # None uses the BF fit
diameter = 0
phases = np.linspace(-np.pi, np.pi, 32) 
#phases = [np.pi/2]

grating = lambda neuron_idx, contrast, orientation,phase: dp.generate_grating(
  *dp.get_grating_params(analyzer.bf_stats, neuron_idx, orientation=orientation,
  phase=phase, contrast=contrast, diameter=diameter), gauss_sigma)

phase_gratings = np.stack([grating(neuron_idx, contrast, orientation, phase)
  for neuron_idx in neuron_ids
  for phase in phases], axis=0)


#phase_gratings.shape
grating_fig = pf.plot_weights(phase_gratings, figsize=(8,8))
weights = analyzer.evals["lca_subspace/weights/w:0"].T
weights_fig = pf.plot_weights(weights[:16, :].reshape((16, 16, 16)), figsize=(8,8))

In [ ]:
model_eval = analyzer.evaluate_model(phase_gratings.reshape(len(phase_gratings), 256),
  ["inference/activity:0", "output/reconstruction:0"])
recon_patches = model_eval["output/reconstruction:0"]
a_vals = model_eval["inference/activity:0"]

In [ ]:
with tf.Session(graph=analyzer.model.graph) as sess:
    sess.run(analyzer.model.init_op)
    feed_dict = analyzer.model.get_feed_dict(phase_gratings.reshape(len(phase_gratings), 256), is_test=True)
    group_act, group_angle, group_encodings = sess.run([analyzer.model.get_group_activity(), 
                                                        analyzer.model.get_group_angle(), 
                                                        analyzer.model.get_group_encodings()], feed_dict)

## Plot Weights (Phis)

In [ ]:
def plot_group_weights(W, group_size, figsize=(10, 10)):
    num_groups = W.shape[0] // group_size
    assert isinstance(num_groups, int), "number of groups should have type int"
    fig, ax = plt.subplots(ncols=group_size, nrows=num_groups ,figsize=figsize)
    w_i = 0
    for r in range(group_size):
        for c in range(group_size):
            ax[r, c].imshow(W[w_i])
            break
            w_i += 1
        break
    plt.show()

In [ ]:
from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(1, 1))
big_gs = GridSpec(1, 2, figure=fig)

gs00 = big_gs[0].subgridspec(2, 2) #gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=big_gs[0])
big_gs[0].update(wspace=0.025, hspace=0.05)
ax00_00 = fig.add_subplot(gs00[0, 0])
ax00_01 = fig.add_subplot(gs00[0, 1])
ax00_10 = fig.add_subplot(gs00[1, 0])
ax00_11 = fig.add_subplot(gs00[1, 1])

gs01 = big_gs[1].subgridspec(2, 2)
ax01_00 = fig.add_subplot(gs01[0, 0])
ax01_01 = fig.add_subplot(gs01[0, 1])
ax01_10 = fig.add_subplot(gs01[1, 0])
ax01_11 = fig.add_subplot(gs01[1, 1])

for i, ax in enumerate(fig.axes):
    ax.imshow(weights_reshaped[i])
    ax.tick_params("both",
                   bottom=False,
                   left=False,
                   labelbottom=False,
                   labelleft=False)
                    

In [ ]:
group_ids = [[j+i for i in range(4)] for j in np.arange(0, analyzer.model_params.num_neurons, 4)]
pf.plot_group_weights(weights, group_ids)

In [ ]:
test = weights_reshaped[:16]

In [ ]:
group_size = analyzer.model_params.num_neurons // analyzer.model_params.num_groups
fig, ax = plt.subplots(nrows=9, ncols=128//8, figsize=(4, 8))
k = 0
for i in range(9):
    for j in range(128//8):
        if i == 4:
            ax[i, j].imshow(np.zeros(shape=(16, 1)), cmap="binary")
            ax[i, j].axis("off")
        else:
            ax[i, j].imshow(weights_reshaped[k], cmap="gray")
            ax[i, j].axis("off")
            k += 1
plt.subplots_adjust(wspace=0.05, hspace=0.025)

In [ ]:
plt.imshow(np.ones(shape=(5, 2)))

In [ ]:
x = np.arange(4*10*256).reshape(4)

In [ ]:
group_size = analyzer.model_params.num_neurons // analyzer.model_params.num_groups
plot_group_weights(weights_reshaped, group_size)

## Reconstruction of Full Image

In [ ]:
log_names = [name[17:-4] for name in os.listdir(os.path.expanduser("~/Work/Projects/lca_subspace_vh/logfiles"))]
log_names.sort()
log_names = log_names[1:]
log_names.pop(2)
filtered_log_names = log_names
print(filtered_log_names)

class params:
    def __init__(self, version):
        self.model_type = "lca_subspace"
        self.model_name = "lca_subspace_vh"
#         self.version = "4_3.0_0.3"
        self.version = version
        self.save_info = "analysis_train"
        self.overwrite_analysis_log = False
        
for v in filtered_log_names[13:]:
    print(v)
    analysis_params = params(v)
    analysis_params.project_dir = (os.path.expanduser("~")+"/Work/Projects/")
    analysis_params.model_dir = (analysis_params.project_dir+analysis_params.model_name)
    
    analyzer = ap.get_analyzer(analysis_params.model_type)
    analyzer.setup(analysis_params)
    analyzer.setup_model(analyzer.model_params)
    analyzer.load_analysis(save_info=analysis_params.save_info)
    analyzer.model_name = analysis_params.model_name

#     dataset = ds.get_data(analyzer.model_params)
#     dataset = analyzer.model.preprocess_dataset(dataset, analyzer.model_params)
#     dataset = analyzer.model.reshape_dataset(dataset, analyzer.model_params)

#     analyzer.model.num_groups = int(v[0])
    
#     keys=["a_fraction_active", "recon_loss", "sparse_loss", "total_loss"]
#     labels=["activity", "recon loss", "sparse loss", "total loss"]
#     stats_fig = pf.plot_stats(analyzer.run_stats, keys=keys, labels=labels, start_index=100, figsize=(10,10))
#     stats_fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_train_stats.png")

    try:
        normed_image = (
          (analyzer.full_image - np.min(analyzer.full_image))
          / (np.max(analyzer.full_image) - np.min(analyzer.full_image))).astype(np.float32)

        normed_recon = (
          (analyzer.full_recon - np.min(analyzer.full_recon))
          / (np.max(analyzer.full_recon) - np.min(analyzer.full_recon))).astype(np.float32)

        fig, ax = plt.subplots(1, 2, figsize=(12,12))
        ax[0] = pf.clear_axis(ax[0])
        ax[0].imshow(np.squeeze(normed_image), cmap="Greys_r")
        ax[0].set_title("Input Image", fontsize=16)
        ax[1] = pf.clear_axis(ax[1])
        ax[1].imshow(np.squeeze(normed_recon), cmap="Greys_r")
        percent_active = "{:.2f}".format(analyzer.recon_frac_act*100)
        psnr = "{:.2f}".format(compare_psnr(normed_image, normed_recon, data_range=1))
        ax[1].set_title("Reconstruction\n"+percent_active+" percent active"+"\n"+"PSNR = "+psnr, fontsize=16)
        plt.show()
        fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_image_recon.png", transparent=True,
          bbox_inches="tight")
    except:
        print(v, "error")
        continue
    

In [ ]:
if analyzer.model_params.whiten_data:
  # FT method is the only one that works on full images
  wht_img, img_mean, ft_filter = dp.whiten_data(full_image,
    method="FT", lpf_cutoff=analyzer.model_params.lpf_cutoff)
else:
  wht_img = full_image
img_patches = dp.extract_patches(wht_img,
  out_shape=(1, analyzer.model_params.patch_edge_size, analyzer.model_params.patch_edge_size, 1),
  overlapping=False, randomize=False, var_thresh=0.0)
img_patches, orig_shape = dp.reshape_data(img_patches, flatten=True)[:2]
model_eval = analyzer.evaluate_model(img_patches,
  ["inference/activity:0", "output/reconstruction:0"])
recon_patches = model_eval["output/reconstruction:0"]
a_vals = model_eval["inference/activity:0"]
recon_patches = dp.reshape_data(recon_patches, flatten=False, out_shape=orig_shape)[0]
full_recon = dp.patches_to_image(recon_patches, full_image.shape).astype(np.float32)
if analyzer.model_params.whiten_data:
    full_recon = dp.unwhiten_data(full_recon, img_mean, ft_filter, method="FT")

In [ ]:
normed_image = (
  (analyzer.full_image - np.min(analyzer.full_image))
  / (np.max(analyzer.full_image) - np.min(analyzer.full_image))).astype(np.float32)

normed_recon = (
  (analyzer.full_recon - np.min(analyzer.full_recon))
  / (np.max(analyzer.full_recon) - np.min(analyzer.full_recon))).astype(np.float32)

fig, ax = plt.subplots(1, 2, figsize=(12,12))
ax[0] = pf.clear_axis(ax[0])
ax[0].imshow(np.squeeze(normed_image), cmap="Greys_r")
ax[0].set_title("Input Image", fontsize=16)
ax[1] = pf.clear_axis(ax[1])
ax[1].imshow(np.squeeze(normed_recon), cmap="Greys_r")
percent_active = "{:.2f}".format(analyzer.recon_frac_act*100)
psnr = "{:.2f}".format(compare_psnr(normed_image, normed_recon, data_range=1))
ax[1].set_title("Reconstruction\n"+percent_active+" percent active"+"\n"+"PSNR = "+psnr, fontsize=16)
plt.show()
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_image_recon.png", transparent=True,
  bbox_inches="tight")

## Location Frequency Terms

In [ ]:
fig = pf.plot_loc_freq_summary(analyzer.bf_stats, figsize=(12, 4), fontsize=16)
fig.savefig(analyzer.analysis_out_dir+"/vis/fig_location_frequency_centers.png")

In [ ]:
keys=["a_fraction_active", "recon_loss", "sparse_loss", "total_loss"]
labels=["activity", "recon loss", "sparse loss", "total loss"]
stats_fig = pf.plot_stats(analyzer.run_stats, keys=keys, labels=labels, start_index=100, figsize=(10,10))
stats_fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_train_stats.png")

In [ ]:
ot_fig = pf.plot_contrast_orientation_tuning(analyzer.ot_grating_responses["neuron_indices"],
  analyzer.ot_grating_responses["contrasts"],
  analyzer.ot_grating_responses["orientations"],
  analyzer.ot_grating_responses["mean_responses"])
ot_fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_orientation_tuning.pdf")

# iso-response

In [ ]:
vector_id = 4
analyzer.bf_id0 = 50#vectors[vector_id, 0] # RICA 83, 85; vae 4, 72; lca 3, 588
analyzer.bf_id1 = 10#vectors[vector_id, 1]
fig, ax = plt.subplots(2)
ax[0] = pf.clear_axis(ax[0])
ax[0].imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id0], cmap="Greys_r")
ax[0].set_title(str(analyzer.bf_id0))
ax[1] = pf.clear_axis(ax[1])
ax[1].imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id1], cmap="Greys_r")
ax[1].set_title(str(analyzer.bf_id1))
plt.show()

In [ ]:
analyzer.bf0 = analyzer.bf_stats["basis_functions"][analyzer.bf_id0].reshape(
(analyzer.model_params.num_pixels))
analyzer.bf1 = analyzer.bf_stats["basis_functions"][analyzer.bf_id1].reshape(
(analyzer.model_params.num_pixels))
analyzer.bf0_norm = np.linalg.norm(analyzer.bf0)
analyzer.bf1_norm = np.linalg.norm(analyzer.bf1)
analyzer.bf0 = analyzer.bf0 / analyzer.bf0_norm
analyzer.bf1 = analyzer.bf1 / analyzer.bf1_norm

print("BF indices = [",analyzer.bf_id0,", ",analyzer.bf_id1,"]")
print("bf0 norm = ", analyzer.bf0_norm)
print("bf1 norm = ", analyzer.bf1_norm)

In [ ]:
num_imgs = int(228**2)
x_pts = np.linspace(-2.0, 2.0, int(np.sqrt(num_imgs)))
y_pts = np.linspace(-2.0, 2.0, int(np.sqrt(num_imgs)))
X_mesh, Y_mesh = np.meshgrid(x_pts, y_pts)
proj_datapoints = np.stack([X_mesh.reshape(num_imgs), Y_mesh.reshape(num_imgs)], axis=1)

analyzer.proj_datapoints = proj_datapoints
analyzer.proj_matrix, v = analyzer.bf_projections(analyzer.bf0, analyzer.bf1)
analyzer.proj_neuron0 = np.dot(analyzer.proj_matrix, analyzer.bf0).T
analyzer.proj_neuron1 = np.dot(analyzer.proj_matrix, analyzer.bf1).T
analyzer.proj_v = np.dot(analyzer.proj_matrix, v).T

analyzer.datapoints = np.stack([np.dot(analyzer.proj_matrix.T, analyzer.proj_datapoints[data_id,:])
for data_id in range(num_imgs)]) #inject
analyzer.datapoints, orig_shape = dp.reshape_data(analyzer.datapoints, flatten=False)[:2]
analyzer.datapoints = {"test": Dataset(analyzer.datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
params={"whiten_data":analyzer.model_params.whiten_data}
if params["whiten_data"]:
    params["whiten_method"] = analyzer.model_params.whiten_method
analyzer.datapoints = analyzer.model.preprocess_dataset(analyzer.datapoints, params=params)
analyzer.datapoints = analyzer.model.reshape_dataset(analyzer.datapoints, analyzer.model_params)
analyzer.datapoints["test"].images /= np.max(np.abs(analyzer.datapoints["test"].images))
analyzer.datapoints["test"].images *= 3*analyzer.analysis_params.input_scale

In [ ]:
activations = analyzer.compute_activations(analyzer.datapoints["test"].images)
#activations = analyzer.evaluate_model(analyzer.datapoints["test"].images, [analyzer.model.module.u_list[1].name])[analyzer.model.module.u_list[1].name]
activity_max = np.amax(np.abs(activations)) # Rescale between -1 and 1
analyzer.norm_activity = activations / (activity_max + 0.00001)

In [ ]:
num_plots_y = 1
num_plots_x = 2
gs1 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=0.5, width_ratios=[4, 1])
fig = plt.figure(figsize=(6,6))
curve_ax = pf.clear_axis(fig.add_subplot(gs1[0]))
#cmap = plt.get_cmap('tab20b')
cmap = plt.get_cmap('viridis')
vmin = np.floor(np.min(analyzer.norm_activity))#0.0
vmax = np.ceil(np.max(analyzer.norm_activity))#1.0

#name_suffix = "continuous"
#pts = curve_ax.scatter(analyzer.proj_datapoints[:,0], analyzer.proj_datapoints[:,1],
#  vmin=vmin, vmax=vmax, cmap=cmap, alpha=0.5, c=analyzer.norm_activity[:, analyzer.bf_id0], s=5.0)

levels = 5
name_suffix = ""
contsf = curve_ax.contourf(X_mesh, Y_mesh,
analyzer.norm_activity[:,analyzer.bf_id0].reshape(int(np.sqrt(num_imgs)),
int(np.sqrt(num_imgs))), levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)

curve_ax.arrow(0, 0, analyzer.proj_neuron0[0].item(),
analyzer.proj_neuron0[1].item(), width=0.05, head_width=0.15,
head_length=0.15, fc='r', ec='r')
curve_ax.arrow(0, 0, analyzer.proj_neuron1[0].item(),
analyzer.proj_neuron1[1].item(), width=0.05, head_width=0.15,
head_length=0.15, fc='w', ec='w')
#curve_ax.arrow(0, 0, analyzer.proj_v[0].item(), analyzer.proj_v[1].item(), width=0.05, head_width=0.15,
# head_length=0.15, linestyle="-.", fc='k', ec='k')

#curve_ax.set_title("Angle = "+"{:.2f}".format(analyzer.neuron_angles[bf_id0, bf_id1])+" deg", fontsize=16)
curve_ax.set_ylim([-2, 2.0])
curve_ax.set_xlim([-2, 2.0])
curve_ax.set_aspect("equal")
#cbar = pf.add_colorbar_to_im(pts, aspect=20, pad_fraction=0.5, labelsize=16, ticks=[vmin, vmax])
#cbar.ax.set_yticklabels(["{:.0f}".format(vmin), "{:.0f}".format(vmax)])

gs2 = gridspec.GridSpecFromSubplotSpec(2, 1, gs1[1], wspace=2, hspace=-0.55)
bf1_ax = pf.clear_axis(fig.add_subplot(gs2[0]))
bf1_ax.imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id0], cmap="Greys_r")
bf1_ax.set_title("Primary\nBasis Function", color='r', fontsize=16)
bf2_ax = pf.clear_axis(fig.add_subplot(gs2[1]))
bf2_ax.imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id1], cmap="Greys_r")
bf2_ax.set_title("Comparison\nBasis Function", color='k', fontsize=16)
fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_response_contours_bf0id"+str(analyzer.bf_id0)+"_bf1id"+str(analyzer.bf_id1)+name_suffix+".png",
transparent=True, bbox_inches="tight", pad_inches=0.01)
plt.show()

In [ ]:
I = np.arange(10)*10
for i in np.arange(0, 100, 4):
    for j in np.arange(4):
        if i == j:
            continue
        try:
            print("working on ", i, i+j)
            vector_id = 4
            analyzer.bf_id0 = i#50#vectors[vector_id, 0] # RICA 83, 85; vae 4, 72; lca 3, 588
            analyzer.bf_id1 = i+j#10#vectors[vector_id, 1]
        #     fig, ax = plt.subplots(2)
        #     ax[0] = pf.clear_axis(ax[0])
        #     ax[0].imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id0], cmap="Greys_r")
        #     ax[0].set_title(str(analyzer.bf_id0))
        #     ax[1] = pf.clear_axis(ax[1])
        #     ax[1].imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id1], cmap="Greys_r")
        #     ax[1].set_title(str(analyzer.bf_id1))
        #     plt.show()

            analyzer.bf0 = analyzer.bf_stats["basis_functions"][analyzer.bf_id0].reshape(
            (analyzer.model_params.num_pixels))
            analyzer.bf1 = analyzer.bf_stats["basis_functions"][analyzer.bf_id1].reshape(
            (analyzer.model_params.num_pixels))
            analyzer.bf0_norm = np.linalg.norm(analyzer.bf0)
            analyzer.bf1_norm = np.linalg.norm(analyzer.bf1)
            analyzer.bf0 = analyzer.bf0 / analyzer.bf0_norm
            analyzer.bf1 = analyzer.bf1 / analyzer.bf1_norm

            print("BF indices = [",analyzer.bf_id0,", ",analyzer.bf_id1,"]")
            print("bf0 norm = ", analyzer.bf0_norm)
            print("bf1 norm = ", analyzer.bf1_norm)

            num_imgs = int(228**2)
            x_pts = np.linspace(-2.0, 2.0, int(np.sqrt(num_imgs)))
            y_pts = np.linspace(-2.0, 2.0, int(np.sqrt(num_imgs)))
            X_mesh, Y_mesh = np.meshgrid(x_pts, y_pts)
            proj_datapoints = np.stack([X_mesh.reshape(num_imgs), Y_mesh.reshape(num_imgs)], axis=1)

            analyzer.proj_datapoints = proj_datapoints
            analyzer.proj_matrix, v = analyzer.bf_projections(analyzer.bf0, analyzer.bf1)
            analyzer.proj_neuron0 = np.dot(analyzer.proj_matrix, analyzer.bf0).T
            analyzer.proj_neuron1 = np.dot(analyzer.proj_matrix, analyzer.bf1).T
            analyzer.proj_v = np.dot(analyzer.proj_matrix, v).T

            analyzer.datapoints = np.stack([np.dot(analyzer.proj_matrix.T, analyzer.proj_datapoints[data_id,:])
            for data_id in range(num_imgs)]) #inject
            analyzer.datapoints, orig_shape = dp.reshape_data(analyzer.datapoints, flatten=False)[:2]
            analyzer.datapoints = {"test": Dataset(analyzer.datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
            params={"whiten_data":analyzer.model_params.whiten_data}
            if params["whiten_data"]:
                params["whiten_method"] = analyzer.model_params.whiten_method
            analyzer.datapoints = analyzer.model.preprocess_dataset(analyzer.datapoints, params=params)
            analyzer.datapoints = analyzer.model.reshape_dataset(analyzer.datapoints, analyzer.model_params)
            analyzer.datapoints["test"].images /= np.max(np.abs(analyzer.datapoints["test"].images))
            analyzer.datapoints["test"].images *= 3*analyzer.analysis_params.input_scale

            activations = analyzer.compute_activations(analyzer.datapoints["test"].images)
            #activations = analyzer.evaluate_model(analyzer.datapoints["test"].images, [analyzer.model.module.u_list[1].name])[analyzer.model.module.u_list[1].name]
            activity_max = np.amax(np.abs(activations)) # Rescale between -1 and 1
            analyzer.norm_activity = activations / (activity_max + 0.00001)

            num_plots_y = 1
            num_plots_x = 2
            gs1 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=0.5, width_ratios=[4, 1])
            fig = plt.figure(figsize=(6,6))
            curve_ax = pf.clear_axis(fig.add_subplot(gs1[0]))
            #cmap = plt.get_cmap('tab20b')
            cmap = plt.get_cmap('viridis')
            vmin = np.floor(np.min(analyzer.norm_activity))#0.0
            vmax = np.ceil(np.max(analyzer.norm_activity))#1.0

            #name_suffix = "continuous"
            #pts = curve_ax.scatter(analyzer.proj_datapoints[:,0], analyzer.proj_datapoints[:,1],
            #  vmin=vmin, vmax=vmax, cmap=cmap, alpha=0.5, c=analyzer.norm_activity[:, analyzer.bf_id0], s=5.0)

            levels = 5
            name_suffix = ""
            contsf = curve_ax.contourf(X_mesh, Y_mesh,
            analyzer.norm_activity[:,analyzer.bf_id0].reshape(int(np.sqrt(num_imgs)),
            int(np.sqrt(num_imgs))), levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)

            curve_ax.arrow(0, 0, analyzer.proj_neuron0[0].item(),
            analyzer.proj_neuron0[1].item(), width=0.05, head_width=0.15,
            head_length=0.15, fc='r', ec='r')
            curve_ax.arrow(0, 0, analyzer.proj_neuron1[0].item(),
            analyzer.proj_neuron1[1].item(), width=0.05, head_width=0.15,
            head_length=0.15, fc='w', ec='w')
            #curve_ax.arrow(0, 0, analyzer.proj_v[0].item(), analyzer.proj_v[1].item(), width=0.05, head_width=0.15,
            # head_length=0.15, linestyle="-.", fc='k', ec='k')

            #curve_ax.set_title("Angle = "+"{:.2f}".format(analyzer.neuron_angles[bf_id0, bf_id1])+" deg", fontsize=16)
            curve_ax.set_ylim([-2, 2.0])
            curve_ax.set_xlim([-2, 2.0])
            curve_ax.set_aspect("equal")
            #cbar = pf.add_colorbar_to_im(pts, aspect=20, pad_fraction=0.5, labelsize=16, ticks=[vmin, vmax])
            #cbar.ax.set_yticklabels(["{:.0f}".format(vmin), "{:.0f}".format(vmax)])

            gs2 = gridspec.GridSpecFromSubplotSpec(2, 1, gs1[1], wspace=2, hspace=-0.55)
            bf1_ax = pf.clear_axis(fig.add_subplot(gs2[0]))
            bf1_ax.imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id0], cmap="Greys_r")
            bf1_ax.set_title("Primary\nBasis Function", color='r', fontsize=16)
            bf2_ax = pf.clear_axis(fig.add_subplot(gs2[1]))
            bf2_ax.imshow(analyzer.bf_stats["basis_functions"][analyzer.bf_id1], cmap="Greys_r")
            bf2_ax.set_title("Comparison\nBasis Function", color='k', fontsize=16)
            fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_response_contours_bf0id"+str(analyzer.bf_id0)+"_bf1id"+str(analyzer.bf_id1)+name_suffix+".png",
            transparent=True, bbox_inches="tight", pad_inches=0.01)
            plt.show()
            
        except:
            print(i, j, "failed to show iso-contour")

In [ ]:
class img_params():
    data_type = analysis_params.data_type
    num_images = 2
    extract_patches = False
    image_edge_size = analysis_params.image_edge_size
    data_dir = os.path.expanduser("~")+"/Work/Datasets/"
    rand_seed = analysis_params.rand_seed
    rand_state = np.random.RandomState(analysis_params.rand_seed)
full_img = dp.reshape_data(ds.get_data(img_params)["train"].images[0], flatten=False)[0]
analyzer.run_patch_recon_analysis(full_img, save_info=analysis_params.save_info)


In [ ]:
def run_patch_recon_analysis(self, full_image, save_info):
    """
    Break image into patches, compute recons, reassemble recons back into a full image
    """
    self.full_image = full_image
    if self.model_params.whiten_data:
      # FT method is the only one that works on full images
      wht_img, img_mean, ft_filter = dp.whiten_data(full_image,
        method="FT", lpf_cutoff=self.model_params.lpf_cutoff)
    else:
      wht_img = full_image
    img_patches = dp.extract_patches(wht_img,
      out_shape=(1, self.model_params.patch_edge_size, self.model_params.patch_edge_size, 1),
      overlapping=False, randomize=False, var_thresh=0.0)
    img_patches, orig_shape = dp.reshape_data(img_patches, flatten=True)[:2]
    model_eval = self.evaluate_model(img_patches,
      ["inference/activity:0", "output/reconstruction:0"])
    recon_patches = model_eval["output/reconstruction:0"]
    a_vals = model_eval["inference/activity:0"]
    self.recon_frac_act = np.array(np.count_nonzero(a_vals) / float(a_vals.size))
    recon_patches = dp.reshape_data(recon_patches, flatten=False, out_shape=orig_shape)[0]
    self.full_recon = dp.patches_to_image(recon_patches, full_image.shape).astype(np.float32)
    if self.model_params.whiten_data:
      self.full_recon = dp.unwhiten_data(self.full_recon, img_mean, ft_filter, method="FT")
    np.savez(self.analysis_out_dir+"savefiles/full_recon_"+save_info+".npz",
      data={"full_image":self.full_image, "full_recon":self.full_recon,
      "recon_frac_act":self.recon_frac_act})
    self.analysis_logger.log_info("Patch recon analysis is complete.")

In [ ]:
def recon_images(analyzer, full_images):
    """
    Paramters: 
        full_images [np.ndarray]: list of images
    """
    model_params = analyzer.model_params
    wht_imgs, img_means, ft_filters = dp.whiten_data(full_image, method="FT", 
                                                  lpf_cutoff=model_params.lpf_cutoff)
    img_patches = dp.extract_patches(wht_imgs, 
                                     out_shape=(1, model_params.patch_edge_size, 
                                                model_params.patch_edge_size, 1),
                                     overlapping=False, randomize=False, var_thresh=0.0)
    img_patches, orig_shape = dp.reshape_data(img_patches, flatten=True)[:2]
    model_eval = analyzer.evaluate_model(img_patches,
      ["inference/activity:0", "output/reconstruction:0"])
    recon_patches = model_eval["output/reconstruction:0"]
    a_vals = model_eval["inference/activity:0"]
    recon_frac_act = np.array(np.count_nonzero(a_vals) / float(a_vals.size))
    recon_patches = dp.reshape_data(recon_patches, flatten=False, out_shape=orig_shape)[0]
    full_recon = dp.patches_to_image(recon_patches, full_image.shape).astype(np.float32)
    if model_params.whiten_data:
          full_recon = dp.unwhiten_data(full_recon, img_mean, ft_filter, method="FT")
    return full_recon, recon_frac_act

In [ ]:
analyzer.model_params.__dict__.keys()

In [ ]:
full_image = np.repeat(analyzer.full_image, 3, axis=0)
full_recon = recon_images(analyzer, full_image)

In [ ]:
full_recon

In [ ]:
model_params = analyzer.model_params
wht_imgs, img_mean, ft_filter = dp.whiten_data(full_image, method="FT", 
                                              lpf_cutoff=model_params.lpf_cutoff)
img_patches = dp.extract_patches(wht_imgs, 
                                 out_shape=(1, model_params.patch_edge_size, 
                                            model_params.patch_edge_size, 1),
                                 overlapping=False, randomize=False, var_thresh=0.0)
img_patches, orig_shape = dp.reshape_data(img_patches, flatten=True)[:2]
model_eval = analyzer.evaluate_model(img_patches,
  ["inference/activity:0", "output/reconstruction:0"])
recon_patches = model_eval["output/reconstruction:0"]
a_vals = model_eval["inference/activity:0"]
recon_frac_act = np.array(np.count_nonzero(a_vals) / float(a_vals.size))
recon_patches = dp.reshape_data(recon_patches, flatten=False, out_shape=orig_shape)[0]
full_recon = dp.patches_to_image(recon_patches, full_image.shape).astype(np.float32)
if model_params.whiten_data:
  full_recon = dp.unwhiten_data(full_recon, img_mean, ft_filter, method="FT")

In [ ]:
def recon_images(analyzer, full_images):
    if analyzer.model_params.whiten_data:
      # FT method is the only one that works on full images
      wht_img, img_mean, ft_filter = dp.whiten_data(full_image,
        method="FT", lpf_cutoff=analyzer.model_params.lpf_cutoff)
    else:
      wht_img = full_image
    img_patches = dp.extract_patches(wht_img,
      out_shape=(1, analyzer.model_params.patch_edge_size, analyzer.model_params.patch_edge_size, 1),
      overlapping=False, randomize=False, var_thresh=0.0)
    img_patches, orig_shape = dp.reshape_data(img_patches, flatten=True)[:2]
    model_eval = analyzer.evaluate_model(img_patches,
      ["inference/activity:0", "output/reconstruction:0"])
    recon_patches = model_eval["output/reconstruction:0"]
    a_vals = model_eval["inference/activity:0"]
    recon_frac_act = np.array(np.count_nonzero(a_vals) / float(a_vals.size))
    recon_patches = dp.reshape_data(recon_patches, flatten=False, out_shape=orig_shape)[0]
    full_recon = dp.patches_to_image(recon_patches, full_image.shape).astype(np.float32)
    if analyzer.model_params.whiten_data:
      full_recon = dp.unwhiten_data(full_recon, img_mean, ft_filter, method="FT")
    return full_recon, recon_frac_act

In [ ]:
normed_image = (
  (full_image - np.min(full_image))
  / (np.max(full_image) - np.min(full_image))).astype(np.float32)

normed_recon = (
  (full_recon - np.min(full_recon))
  / (np.max(full_recon) - np.min(full_recon))).astype(np.float32)

group_size = analyzer.model_params.num_neurons // analyzer.model_params.num_groups

In [ ]:
a_vals.shape

In [ ]:
full_recon.shape

In [ ]:
full_recon

In [ ]:
analyzer.model_params.__dict__

In [ ]:
with tf.Session() as sess:
    x = np.arange(50).reshape(5, 10)
    print(x)
    I = tf.slice(x, [3, 0], [1, 10])
    print(sess.run(tf.transpose(I)))

In [ ]:
x = np.zeros(shape=(256, 64))
for i in range(64):
    x[i:(i+1)*4, i] = 1.
x[:8, :]

In [ ]:
x = np.random.rand(100, 100)
with tf.Session() as sess:
    y = tf.matmul(tf.real(tf.linalg.inv(tf.sqrt(tf.matmul(x, tf.transpose(x))))), x)
    print(sess.run(y))

In [ ]:
x = np.random.rand(50, 50)
np.all(x @ x.T > 0)

In [ ]:
ot_fig = pf.plot_contrast_orientation_tuning(analyzer.ot_grating_responses["neuron_indices"],
  analyzer.ot_grating_responses["contrasts"],
  analyzer.ot_grating_responses["orientations"],
  analyzer.ot_grating_responses["mean_responses"],
  figsize=(32,32))
ot_fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_orientation_tuning.pdf")